In [ ]:
pip install snowflake-connector-python==2.3.6 > /dev/null

In [ ]:
# Get Snowflake credentials
from getpass import getpass
accountName = getpass('Enter Snowflake account name')
userName = getpass('Enter Snowflake user name:')
password = getpass('Enter Snowflake password')
databaseName = 'ipeds' #@param {type:"string"}
sourceSchema = "Public" #@param {type:"string"}
warehouseName = "COMPUTE_WH" #@param {type:"string"}

Enter Snowflake account name··········
Enter Snowflake user name:··········
Enter Snowflake password··········


In [ ]:
# Connect to snowflake and execute the query
# Create dataframe from dataset
# Close all the open connection

import snowflake.connector
import pandas as pd
from __future__ import print_function
from ipywidgets import interactive

import ipywidgets as widgets

#get Snowflake version
ctx = snowflake.connector.connect(
    user=userName,
    password=password,
    account=accountName,
    database=databaseName,
    warehouse=warehouseName
)
cs = ctx.cursor()
try:
  sql = """  
  SELECT 
    EN.AcademicYear  
    ,Inst.State   
    ,SUM(TOTALMEN) AS "Total Men"
    ,SUM(TOTALWOMEN) AS "Total Women"
    ,SUM(EN.WhiteMen) AS "White Men"
    ,SUM(EN.WhiteWomen) AS "White Women"
    ,SUM(EN.NonresidentAlienMen) AS "Non resident alien Men"
    ,SUM(EN.NonresidentAlienWomen) AS "Non resident alien Women"
    ,SUM(EN.AFRICANAMERICANMEN) AS "Black/AfricanAmerican Men"
    ,SUM(EN.AFRICANAMERICANWOMEN) AS "Black/AfricanAmerican Women"
    ,SUM(EN.HispanicOrLatinoMen) AS "Hispanic Men"
    ,SUM(EN.HispanicOrLatinoWomen) AS "Hispanic Women"
    ,SUM(EN.AsianMen) AS "Asian Men"
    ,SUM(EN.AsianWomen) AS "Asian Women"
    ,SUM(EN.AmericanIndianOrAlaskaNativeMen) AS "American \
          Indian /Alaska Native Men"
    ,SUM(EN.AmericanIndianOrAlaskaNativeWomen) AS "American \
          Indian /Alaska Native Women"
    ,SUM(EN.HAWAIIANPACIFICISLANDERMEN) AS "Native Hawaiian Men"
    ,SUM(EN.HAWAIIANPACIFICISLANDERWOMEN) 
        AS "Native Hawaiian Women"
  FROM ENROLLMENT EN
  INNER JOIN ACADEMICINSTITUTION Inst 
    ON EN.INSTITUTIONIDENTIFIER = Inst.INSTITUTIONIDENTIFIER
  WHERE Inst.DEGREEGRANTINGSTATUS = 1  
  AND Inst.State IN ('TX','FL','CA','NY','PA','IL','OH','GA','NC'\
      ,'MI','NJ','VA','WA','AZ','MA','TN','IN','MO','WI','MD')
  GROUP BY EN.AcademicYear,Inst.State ;
 """
  cs.execute(sql)
  records = cs.fetchall()
  names = [x[0] for x in cs.description]
  df1 = pd.DataFrame(records,columns=names) 
finally:
    cs.close()
    ctx.close()

In [ ]:
# Convert all the integer columns to float. 
# Float allows to use the formatting while printing dataframe
for (columnName, columnData) in df1.iteritems():
   #print('Colunm Name : ', columnName)
   if (columnName != 'ACADEMICYEAR' and columnName != 'STATE') :
     df1[columnName] = df1[columnName].astype(float)

In [ ]:
  # Select the year for report  
  def f(Year):
    return Year
  
  w = interactive(f,Year = df1.ACADEMICYEAR .unique());
  display(w);

In [ ]:
# Filter data for the year selected by user & drop 
# the data column not used in reporting layer
dfFilter = df1[df1.ACADEMICYEAR.eq(w.result)] 
dfFilter=dfFilter.drop(columns=['ACADEMICYEAR'])

In [ ]:
# Transpose data from rows to column
# The first row has column name , use the column names from first row 
# Create data frame, rename the first column
# Sort the column 1 to n so state codes comes in desired order and print

dft=dfFilter.T
dft.reset_index(level=0, inplace=True)
headers = dft.iloc[0]
#dft.reindex(columns=sorted(dft.iloc[1:]))
new_df  = pd.DataFrame(dft.values[1:], columns=headers)

new_df = new_df.rename(columns = {"STATE":"Segment"}) 

new_columns = [new_df.columns[0]]
new_columns.extend(sorted(new_df.columns[1:]))
new_df = new_df.reindex(columns=new_columns)

pd.options.display.float_format = '{:,.0f}'.format  
from IPython.display import HTML
HTML(new_df.to_html(index=False))